In [1]:
%reload_ext autoreload
%autoreload 2

In [155]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c
import seaborn as sns

In [23]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('../Customized_Components')
from single_pad_transmon_pocket import TransmonPocket_Single
import Transmon_property as trans_p

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [4]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [7]:
import seaborn as sns

In [6]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45
cc = 1664080*u.fF/u.m

In [319]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '20 mm'
design.chips.main.size['size_y'] = '10 mm'

design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

design.variables['trace_width'] = '12.4 um'
cpw_pin_width = 12.4*u.um
design.variables['trace_gap'] = '7 um'
cpw_gap = 7*u.um

# design.chips['main']['material'] = 'Silicon'

gui = MetalGUI(design)

Component-specific variables

In [316]:


qb_options = dict(
pad_pocket_distance_top = '40um',
connection_pads = dict(
    a = dict(loc_W  = 1, 
            loc_H =  1, 
            pad_gap = '10um',
            pad_height = '20um',
            pad_width = '80um',
            pad_cpw_extent =  '10um',
            pocket_rise = '0um',
            cpw_extend = '5um')))

TQ_options = dict(prime_width = design.variables['cpw_width'],
               prime_gap = design.variables['cpw_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '60um',
               coupling_space = '5um',
               open_termination=False,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False)

CPW_options = Dict(trace_width = design.variables['trace_width'],
               trace_gap  = design.variables['trace_gap'],
        total_length='5 mm',
        hfss_wire_bonds = True,
        q3d_wire_bonds = True,
        fillet='30 um',
        lead = dict(start_straight='20um', end_straight = '50um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end')), )

pin_inputs = Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end'))

trans_options = Dict(hfss_wire_bonds = True,
                     q3d_wirebonds = True,
               pin_inputs=Dict(
                 start_pin=Dict(
                     component='TQ1',
                     pin='prime_start'),
                 end_pin=Dict(
                     component='TQ2',
                     pin='prime_end')))


## Single Pad--Verify Design

### Draw the single pad transmon

In [321]:
design.delete_all_components()

In [322]:
design.overwrite_enabled = True

size = 0.25*u.mm
gap = 30*u.um
size = size.to(u.um)
pocket_width = size+2*gap


op = Dict(
    pos_x = '0',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **qb_options)

q1 = TransmonPocket_Single(design,'Q1',options = op)

gui.rebuild()
gui.zoom_on_components(['Q1'])


In [323]:
TQ_options['down_length'] = '40 um'
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.23 mm',
                                             pos_y='0.5 mm',
                                             mirror = True,
                                             coupling_length='120 um', **TQ_options))
TQ1.options['open_termination'] = True

gui.rebuild()
gui.autoscale()



  return lib.buffer(

  return lib.buffer(



In [324]:
def find_cpw_freq(pad_size, offset, extend, buffer = 150*u.um):
    gap1 = 0.055
    gap = 30*u.um
    size = pad_size.to(u.um)
    pocket_width = size+2*gap
    design.delete_component('cpw_1')
    coupling_len = extend
    q1.options['pad_height'] = '{}'.format(size)
    q1.options['pad_width'] = '{}'.format(size)
    q1.options['pocket_width'] = '{}'.format(pocket_width)
    q1.options['connection_pads']['a']['pad_width'] = '{}'.format(coupling_len)
    # TQ1.options['open_termination'] = True
    gui.rebuild()

    anchors = trans_p.anchor_CPW(q1,buffer, gap1, 2,last_offset = offset)
    design.delete_component('cpw_1')


    pin_inputs = Dict(
                start_pin=Dict(component='Q1', pin='a'),
                end_pin=Dict(component='TQ1', pin='second_end'))

    CPW_options['pin_inputs'] = pin_inputs

    qa = RouteMixed(design, 'cpw_1', options = Dict(anchors = anchors, **CPW_options))

    gui.rebuild()
    gui.autoscale()
    
    length = trans_p.find_total_len(qa, q1, TQ1, count_extend=False)
    return (trans_p.find_actual_frequency(length, cpw_pin_width, cpw_gap), length)

In [339]:
freqs = np.arange(4, 6, 0.2)
sizes = [250,235,220,200,182,172,165,150,138,131]
buffers = np.array([189,199,202,205,206,204,205,207,209,208])
offsets = np.array([0.35,0.40,0.42,0.41,0.41,0.42,0.45,0.45,0.457,0.476])

dic = {}
dic['Target Frequencies'] = freqs
dic['Buffer'] = buffers
dic['Offset'] = offsets
dic['Sizes'] = sizes

data = pd.DataFrame(dic)
data

,Target Frequencies,Buffer,Offset,Sizes
0,4.0,189,0.350,250
1,4.2,199,0.400,235
2,4.4,202,0.420,220
3,4.6,205,0.410,200
4,4.8,206,0.410,182
5,5.0,204,0.420,172
6,5.2,205,0.450,165
7,5.4,207,0.450,150
8,5.6,209,0.457,138
9,5.8,208,0.476,131


In [340]:
freqs[4]

4.800000000000001

In [341]:
for n in range(10):
    pad_size = sizes[n]*u.um
    buffer = buffers[n]*u.um
    offset = offsets[n]
    coupling_len = 20*u.um

    wr = find_cpw_freq(pad_size, offset, coupling_len, buffer)[0]
    print('The resonator frequency is', wr, freqs[n]+2.4)


: 

: 

In [305]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


In [303]:
pd.read_csv('Educated_guess0325.csv')

,Unnamed: 0,Target Frequencies,Buffer,Offset,Sizes
0,0,4.0,202,0.4,250
1,1,4.2,204,0.4,230
2,2,4.4,208,0.4,210
3,3,4.6,206,0.4,196
4,4,4.8,206,0.4,184
5,5,5.0,202,0.4,170
6,6,5.2,203,0.4,155
7,7,5.4,205,0.4,140
8,8,5.6,200,0.4,132
9,9,5.8,190,0.4,125


In [302]:
pd.read_csv('Educated_guess0327.csv')

,Unnamed: 0,Target Frequencies,Buffer,Offset,Sizes
0,0,4.0,202,0.400,250
1,1,4.2,200,0.400,235
2,2,4.4,203,0.420,220
3,3,4.6,206,0.410,200
4,4,4.8,207,0.410,182
5,5,5.0,205,0.420,172
6,6,5.2,206,0.450,165
7,7,5.4,208,0.450,150
8,8,5.6,210,0.457,138
9,9,5.8,210,0.476,131


In [314]:

data.to_csv('Educated_Guess0327.csv')

In [152]:
result = pd.read_csv('/Users/wendy/Desktop/Wendy-qiskit-code/data/CPW_QB_freq_only_0325_2.csv')

In [169]:
result[['pad_size(mm)','buffer(um)','target_freq(GHz)','Freq_EPR1(MHz)']]

,pad_size(mm),buffer(um),target_freq(GHz),Freq_EPR1(MHz)
0,0.250,202.0,4.0,4052.461326
1,0.230,204.0,4.2,4246.707225
2,0.210,208.0,4.4,4475.325304
3,0.196,206.0,4.6,4649.307436
4,0.184,206.0,4.8,4815.997915
5,0.170,202.0,5.0,5042.427256
6,0.155,203.0,5.2,5305.466667
7,0.140,205.0,5.4,5613.889100
8,0.132,200.0,5.6,5796.487743
9,0.125,190.0,5.8,5971.615783


In [161]:
#compare qubit frequency w.r.t. the target frequency
sns.relplot(data = result,x = 'target_freq(GHz)', y = 'Freq_EPR1(MHz)',hue = 'Chi2_1(MHz)')
result['Freq_EPR1(MHz)'].values/1000 - result['target_freq(GHz)'].values

array([0.05246133, 0.04670722, 0.0753253 , 0.04930744, 0.01599792,
       0.04242726, 0.10546667, 0.2138891 , 0.19648774, 0.17161578])

In [162]:
#compare resonator frequency w.r.t. the target frequency
sns.relplot(data = result,x = 'target_res_freq(GHz)', y = 'Freq_EPR2(MHz)',hue = 'Chi2_1(MHz)')
result['Freq_EPR2(MHz)'].values/1000 - result['target_res_freq(GHz)'].values

array([-0.04874854, -0.03000159, -0.03977072, -0.04221077, -0.08378603,
       -0.03670722, -0.03422481, -0.03195547, -0.0187805 ,  0.08313761])